<a href="https://colab.research.google.com/github/conglapgit45/QA_PDF_with_RAG/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all required libraries
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
# Import all required libraries
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [6]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=879a0c9fb23d3bc57ef98a171de0c7ec432a3d0237159a309cb7487e5ff1956d
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


In [7]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [9]:
import gradio as gr


def init_model():
    # Initiate LLM using Vicuna model
    nf4_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type = "nf4",
      bnb_4bit_use_double_quant = True,
      bnb_4bit_compute_dtype = torch.bfloat16
    )

    # Initiate model and tokenizer
    MODEL_NAME = "lmsys/vicuna-7b-v1.5"
    model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      quantization_config = nf4_config,
      low_cpu_mem_usage = True
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Integrate the model and tokenizer into a pipeline
    model_pipeline = pipeline(
      "text-generation",
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 512,
      pad_token_id = tokenizer.eos_token_id,
      device_map = "auto"
    )
    llm = HuggingFacePipeline(
      pipeline = model_pipeline,
    )
    return llm

def format_docs(docs) :
    return "\n\n".join(doc.page_content for doc in docs)

def process_file_qa(FILE_PATH, USER_QUESTION):
    global llm
    # Read PDF file
    Loader = PyPDFLoader
    #FILE_PATH = "./Artificial_Intelligence_Definition_and_Background.pdf"
    loader = Loader(FILE_PATH)
    documents = loader.load()

    # Initiate text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    # Initiate instance vectorization
    embedding = HuggingFaceEmbeddings()

    # Initiate database vector
    vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
    retriever = vector_db.as_retriever()

    # Run and test the program
    prompt = hub.pull("rlm/rag-prompt")
    rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
    )
    # USER_QUESTION = "What are applications of AI?"
    output = rag_chain.invoke(USER_QUESTION)
    answer = output.split('Answer:')[1].strip()
    return answer

llm = init_model()


def clear_file():
    return gr.update(value='')

def clear_text():
    return gr.update(value='')

def generate(USER_QUESTION):
  output = rag_chain.invoke(USER_QUESTION)
  answer = output.split('Answer:')[1].strip()
  return answer

# Create a Gradio interface
with gr.Blocks() as demo:
    file_input = gr.File(label="Upload your file")
    file_input.upload(get_vector_db, [file_input])
    file_clear_btn = gr.Button("Clear File")
    file_clear_btn.click(clear_file, [], [file_input])

    text_input = gr.Textbox(label="Enter some text")
    text_clear_btn = gr.Button("Clear Text")
    text_clear_btn.click(clear_text, [], [text_input])

    output = gr.Textbox(label="Output", interactive=False)
    submit_btn = gr.Button("Submit")
    submit_btn.click(process_file_qa, [file_input, text_input], output)



# Launch the interface
demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dce57472ef4cdabd69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dce57472ef4cdabd69.gradio.live


In [ ]:
def process_file(file):
    if file.type == "text/plain":
      Loader = TextLoader
    elif file.type == "application/pdf":
      Loader = PyPDFLoader

    loader = Loader(file.path)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)
    for i, doc in enumerate(docs) :
      doc.metadata["source"] = f"source_{i}"
    return docs


def get_vector_db(file):
    docs = process_file(file)
    cl.user_session.set("docs", docs)
    vector_db = Chroma.from_documents(documents=docs , embedding=embedding)
    return vector_db


def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5", max_new_token: int = 512):
    nf4_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_quant_type = "nf4",
      bnb_4bit_use_double_quant = True,
      bnb_4bit_compute_dtype = torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config = nf4_config,
      low_cpu_mem_usage = True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline(
      "text-generation",
      model =model,
      tokenizer = tokenizer,
      max_new_tokens = max_new_token,
      pad_token_id = tokenizer.eos_token_id,
      device_map ="auto"
    )

    llm = HuggingFacePipeline(
      pipeline = model_pipeline,
    )
    return llm

LLM = get_huggingface_llm()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [4]:
def init_model():
    # Initiate LLM using Vicuna model
    nf4_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type = "nf4",
      bnb_4bit_use_double_quant = True,
      bnb_4bit_compute_dtype = torch.bfloat16
    )

    # Initiate model and tokenizer
    MODEL_NAME = "lmsys/vicuna-7b-v1.5"
    model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      quantization_config = nf4_config,
      low_cpu_mem_usage = True
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Integrate the model and tokenizer into a pipeline
    model_pipeline = pipeline(
      "text-generation",
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 512,
      pad_token_id = tokenizer.eos_token_id,
      device_map = "auto"
    )
    llm = HuggingFacePipeline(
      pipeline = model_pipeline,
    )
    return llm

def format_docs(docs) :
    return "\n\n".join(doc.page_content for doc in docs)

def process_file_qa(FILE_PATH, USER_QUESTION):
    global llm
    # Read PDF file
    Loader = PyPDFLoader
    #FILE_PATH = "./Artificial_Intelligence_Definition_and_Background.pdf"
    loader = Loader(FILE_PATH)
    documents = loader.load()

    # Initiate text splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    # Initiate instance vectorization
    embedding = HuggingFaceEmbeddings()

    # Initiate database vector
    vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
    retriever = vector_db.as_retriever()

    # Run and test the program
    prompt = hub.pull("rlm/rag-prompt")
    rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
    )
    # USER_QUESTION = "What are applications of AI?"
    output = rag_chain.invoke(USER_QUESTION)
    answer = output.split('Answer:')[1].strip()
    return answer

# llm = init_model()


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]